# Lightweight Fine-Tuning Project

# 1. Prepare the Foundation Model

 TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA [Low Rank Adaptation]
* Model: gpt2
* Evaluation approach: Hugging Face Training, Evaluation
* Fine-tuning dataset: sms_spam

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
!pip install -U datasets
!pip install -q "datasets==2.15.0"
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.15.0
    Uninstalling datasets-2.15.0:
      Successfully uninstalled datasets-2.15.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable


## Load and Pre-process

In [1]:
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

# Check sample data
dataset["train"][10]

{'sms': "Yeah I should be able to, I'll text you when I'm ready to meet up\n",
 'label': 0}

Now we are going to process our datasets by converting all the text into tokens for our models.

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Set the padding token to be the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], padding=True,truncation=True,return_tensors="pt"), batched=True
    )
    
# Inspect the available columns in the dataset
tokenized_dataset["train"]

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

## Load Pre-trained Hugging Face Model

In [3]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

model.config.pad_token_id = tokenizer.pad_token_id

# Ensure the model is on CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) 

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.parameters():
    param.requires_grad = True

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Handling pad_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize the model embeddings to include the new token
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [5]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


## Train the model

It's time to train our model. We'll use the Trainer class.

First we'll define a function to compute our accuracy metreic then we make the Trainer.

In this instance, we will fill in some of the training arguments

In [6]:
import torch
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

'''
def compute_metrics(eval_pred):
    print("eval_pred -> ", eval_pred, "  Type ->  ", type(eval_pred))
    
    predictions, labels = eval_pred
    predictions = torch.tensor(predictions).to("cuda")
    labels = torch.tensor(labels).to("cuda")
    
    print("predictions -> ", predictions, "  Type ->  ", type(predictions))
    print("labels -> ", labels, "  Type ->  ", type(labels))
    
    #preds = eval_pred.predictions.argmax(-1)
    #print("preds -> ", preds, "  Type ->  ", type(preds))
    
    # predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions - labels).mean()}
'''
from sklearn import metrics
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {"accuracy": accuracy}
    
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./output",
        # Set the learning rate
         learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.113600,0.059791,0.990135
2,0.022500,0.069767,0.991031


Checkpoint destination directory ./output/checkpoint-2230 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./output/checkpoint-4460 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=4460, training_loss=0.08706759294586866, metrics={'train_runtime': 631.1479, 'train_samples_per_second': 14.13, 'train_steps_per_second': 7.066, 'total_flos': 882610409668608.0, 'train_loss': 0.08706759294586866, 'epoch': 2.0})

## Evaluate the model 
Evaluating the model is as simple as calling the evaluate method on the trainer object. This will run the model on the test set and compute the metrics we specified in the compute_metrics function.

In [7]:
trainer.evaluate()

{'eval_loss': 0.05979146063327789,
 'eval_accuracy': 0.9901345291479821,
 'eval_runtime': 16.5579,
 'eval_samples_per_second': 67.339,
 'eval_steps_per_second': 33.7,
 'epoch': 2.0}

## View the Results
Let's look at a few examples

In [8]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,1,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,1,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",0,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,1,1


# 2.Perform Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Create a PEFT model

In [9]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

from peft import LoraConfig, TaskType
config = LoraConfig( 
            task_type=TaskType.SEQ_CLS, 
            inference_mode=False, 
            r=8,
            lora_alpha=8, 
            lora_dropout=0.1,
            fan_in_fan_out=True
            )

print("LoRA Configuration Details ---> \n", config)

LoRA Configuration Details ---> 
 LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.SEQ_CLS: 'SEQ_CLS'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.1, fan_in_fan_out=True, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)


In [10]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from transformers import AutoModelForSequenceClassification
new_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
lora_model = get_peft_model(new_model, config)

# Check the trainable parameters
lora_model.print_trainable_parameters()

# Ensure the model is on CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_model.to(device)

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 297,984 || all params: 124,737,792 || trainable%: 0.23888830740245906


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_

FEW ADDITIONAL CHECKS

In [11]:
# Check for Invalid Label Index (Most Common Cause for CUDA error)
print(set(tokenized_dataset["train"]["label"]))
print(set(tokenized_dataset["test"]["label"]))

{0, 1}
{0, 1}


In [12]:
# Check for Incorrect Model Output Shape to avoid CUDA error

import torch
for batch in tokenized_dataset["train"]:
    input_tokens = tokenizer(batch['sms'], return_tensors="pt").to("cuda")
    with torch.no_grad():
        logits = lora_model(**input_tokens).logits
        print("logits.shape", logits.shape)
        break

logits.shape torch.Size([1, 2])


In [13]:
# Check Label Type Mismatch (Subtle Error) - to avoid CUDA error
for batch in tokenized_dataset["train"]:
    print(type(batch["label"]))
    break
    
# Convert labels to int64    
import torch
tokenized_dataset["train"] = tokenized_dataset["train"].map(lambda x: {"label": torch.tensor(x["label"], dtype=torch.long)})
tokenized_dataset["test"] = tokenized_dataset["test"].map(lambda x: {"label": torch.tensor(x["label"], dtype=torch.long)})    
for batch in tokenized_dataset["train"]:
    print(type(batch["label"]))
    break

<class 'int'>
<class 'int'>


## Training/fine-tuning the PEFT model

In [ ]:
# Handle cuda related issues
import os
os.environ["TORCH_USE_CUDA_DSA"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

from sklearn import metrics
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {"accuracy": accuracy}

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./lora_output",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

## Evaluate the lora model

In [12]:
trainer.evaluate()

{'eval_loss': 0.9446949362754822,
 'eval_accuracy': 0.8708520179372198,
 'eval_runtime': 20.0312,
 'eval_samples_per_second': 55.663,
 'eval_steps_per_second': 55.663,
 'epoch': 1.0}

In [13]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,0,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,0,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",0,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,1


## Save the LoRA model

In [14]:
lora_model.save_pretrained("./lora_output")

# 3. Inference using the PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [15]:
from peft import PeftModel, PeftConfig, AutoPeftModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

saved_config = PeftConfig.from_pretrained("./lora_output")
print(saved_config)

LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='gpt2', revision=None, task_type='SEQ_CLS', inference_mode=True, r=8, target_modules=['c_attn'], lora_alpha=8, lora_dropout=0.1, fan_in_fan_out=True, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)


In [16]:
saved_model = AutoPeftModelForSequenceClassification.from_pretrained("./lora_output")
saved_model

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_

In [17]:
tokenizer = AutoTokenizer.from_pretrained(saved_config.base_model_name_or_path)

In [18]:
# Cross check --- 
#https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification

x = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

print(x)

for i in x:
    print(i['input_ids'])
    print(i)
    break

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})
[56, 929, 986, 14690, 788, 530, 1110, 319, 32803, 356, 460, 1265, 21504, 10247, 290, 474, 72, 323, 259, 1011, 2666, 467, 479, 3301, 2088, 220, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [19]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd
import torch

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

predictions = []
for i in items_for_manual_review:
    input_tokens = tokenizer(i['sms'], return_tensors="pt")
    with torch.no_grad():
        logits = model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        predictions.append(predicted_class_id)
print("Predictions ---> \n", predictions)

Predictions ---> 
 [0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        #"predictions": results.predictions.argmax(axis=1),
        "predictions": predictions,
        #"labels": results.label_ids,
        "labels": [item["label"] for item in items_for_manual_review],
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,0,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,0,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",0,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,1


## Evaluate the saved model

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {"accuracy": accuracy}

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=saved_model,
    args=TrainingArguments(
        output_dir="./new_output",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()